In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy    

In [14]:
ROWS = 64
COLS = 64
CHANNELS = 3

TRAIN_DIR = 'C:/Users/Rohan/OneDrive/Desktop/data/Train_data/'
TEST_DIR = 'C:/Users/Rohan/OneDrive/Desktop/data/Test_data/'

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [15]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [16]:
def prepare_data(images):
    m = len(images)
    X = np.zeros((m, ROWS, COLS, CHANNELS), dtype=np.uint8)
    y = np.zeros((1, m))
    for i, image_file in enumerate(images):
        X[i,:] = read_image(image_file)
        if 'dog' in image_file.lower():
            y[0, i] = 1
        elif 'cat' in image_file.lower():
            y[0, i] = 0
    return X, y


In [17]:
def sigmoid(x):
    return 1/( 1+ np.exp(-x))

def sigmoid_der(x):
    return x*(1-x)

In [32]:
#initialising parameters as a matrix of suitable dimensions 
def init_para( n_x, n_h, n_y):
    w1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    
    w2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1":w1,
                  "b1":b1,
                  "W2":w2,
                  "b2":b2}
    return parameters

In [33]:
def forw_prop(X, parameters):
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    
    #following basic formulae for forw prop
    z1 = np.dot(w1, X) + b1
    #normalisation using tanh function
    a1 = np.tanh(z1)
    
    
    z2 = np.dot(w2, a1) + b2
    #normalisation using sigmoid function
    a2 = sigmoid(z2)
    
    temp = {"z1":z1,
            "a1":a1,
            "z2":z2,
            "a2":a2}
    return a2, temp

In [34]:
def what_is_cost(a2, Y, parameters):
    #gives number of rows of output layer in m
    m = Y.shape[1]
    
    #cost is calculated using some complicated looking formula
    #cost basically gives us how good/bad the model has got after training iterations
    #basically calcs error compared to the Y
    v = np.multiply(np.log(a2), Y) + np.multiply((1-Y),np.log(1-a2))
    
    cost = (-1/m)*np.sum(v)
    
    return cost

In [45]:
def back_prop(parameters, temp, X, Y):
    m = X.shape[1]
    w1 = parameters["W1"]
    w2 = parameters["W2"]
    a1 = temp["a1"]
    a2 = temp["a2"]
    
    
    
    #following is calc using set of mathematical formulae attached in the README.file
    #explaination
    dz2 = a2 - Y
    dw2 = (1/m)*np.dot(dz2, a1.T) 
    db2 = (1/m)*np.sum(dz2, axis = 1, keepdims = True)
    dz1 = np.dot( w2.T, dz2)*(1 - np.power(a1,2))
    dw1 = (1/m)*np.dot(dz1, X.T)
    db1 = (1/m)*np.sum(dz1, axis = 1, keepdims = True)
    
 
    
    gradient = {"dW1":dw1,
                 "db1":db1,
                 "dW2":dw2,
                 "db2":db2}
    
    return gradient

In [36]:
def update_parameters(parameters, gradient, alpha ):
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dw1 = gradient["dW1"]
    db1 = gradient["db1"]
    dw2 = gradient["dW2"]
    db2 = gradient["db2"]
    
    w1 = w1 - dw1*alpha
    b1 = b1 - db1*alpha
    w2 = w2 - dw2*alpha
    b2 = b2 - db2*alpha
    
    new_param = {"W1":w1,
                  "b1":b1,
                  "W2":w2,
                  "b2":b2}
    return new_param

In [53]:
def predict(parameters, X):
    m = X.shape[1]
    Y_predict = np.zeros((1,m))
    a2, temp = forw_prop(X, parameters)
    

    
    
    for i in range(a2.shape[1]):
        #converting outputs to predictions
        if a2[0,i]> 0.5:
            Y_predict[[0],[i]] = 1
        else:
            Y_predict[[0],[i]] = 0
            
    return Y_predict       

In [38]:
def model(X_train, Y_train, X_test, Y_test, n_h, num_iterations=1000, alpha = 0.05, print_cost = False):
    
    #size n_x, n_y are taken from the training data
    #dimensions of input and output layers
    n_x = X_train.shape[0]
    n_y = Y_train.shape[0]
    
    #initialising a random value for parameters w1, b1, w2, b2
    #parameters is a dictionary of W1, b1, W2, b2
    parameters = init_para( n_x, n_h, n_y)
    
    #extracting values out of dictionary
    w1 = parameters["W1"]
    b1 = parameters["b1"]
    w2 = parameters["W2"]
    b2 = parameters["b2"]
    
    
    for i in range( num_iterations):
        
        #forward propogation executed to get a2 = output layer
        #ouptut is calc using training data X_train, and random initial params
        #cahce stores a1,z1,a2,z2 in dict
        a2, cache = forw_prop(X_train, parameters )
        
        
        cost = what_is_cost(a2, Y_train, parameters )
        
        #back_prop returns a dictionary of dw1, db1, dw2, db2
        #which are the errors in the initial weights and biases
        gradient = back_prop(parameters, cache, X_train, Y_train)
        
        #update params takes parameters and gradients, and corrects 
        #each parameter according to the learning rate alpha
        parameters = update_parameters(parameters, gradient, alpha)
        
        if( i%200 == 0):
            print("cost after  itrn", i, cost, sep=" " , end="\n")
            
            
        Y_prediction_test = predict(parameters, X_test)
        Y_prediction_train = predict( parameters, X_train) 
        
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

In [39]:
train_set_x, train_set_y = prepare_data(train_images)
test_set_x, test_set_y = prepare_data(test_images)

In [40]:
train_set_x_flatten = train_set_x.reshape(train_set_x.shape[0], ROWS*COLS*CHANNELS).T
test_set_x_flatten = test_set_x.reshape(test_set_x.shape[0], -1).T

In [41]:
train_set_x = train_set_x_flatten/255
test_set_x = test_set_x_flatten/255

In [54]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, n_h=10 , num_iterations = 3000, alpha = 0.003, print_cost = True)


cost after  itrn 0 0.693044197549506
train accuracy: 51.09963345551483 %
test accuracy: 52.6 %
train accuracy: 51.06631122959014 %
test accuracy: 52.7 %
train accuracy: 51.03298900366545 %
test accuracy: 52.7 %
train accuracy: 50.99966677774076 %
test accuracy: 52.800000000000004 %
train accuracy: 51.08297234255249 %
test accuracy: 52.800000000000004 %
train accuracy: 51.09963345551483 %
test accuracy: 52.800000000000004 %
train accuracy: 51.08297234255249 %
test accuracy: 52.900000000000006 %
train accuracy: 51.16627790736421 %
test accuracy: 52.6 %
train accuracy: 51.18293902032656 %
test accuracy: 52.6 %
train accuracy: 51.26624458513829 %
test accuracy: 52.5 %
train accuracy: 51.24958347217594 %
test accuracy: 52.5 %
train accuracy: 51.29956681106298 %
test accuracy: 52.6 %
train accuracy: 51.34955014995002 %
test accuracy: 52.400000000000006 %
train accuracy: 51.38287237587471 %
test accuracy: 52.400000000000006 %
train accuracy: 51.366211262912365 %
test accuracy: 52.2 %
train ac

train accuracy: 52.16594468510497 %
test accuracy: 53.3 %
train accuracy: 52.182605798067314 %
test accuracy: 53.3 %
train accuracy: 52.14928357214262 %
test accuracy: 53.3 %
train accuracy: 52.19926691102966 %
test accuracy: 53.199999999999996 %
train accuracy: 52.23258913695435 %
test accuracy: 53.199999999999996 %
train accuracy: 52.2492502499167 %
test accuracy: 53.199999999999996 %
train accuracy: 52.28257247584139 %
test accuracy: 53.199999999999996 %
train accuracy: 52.28257247584139 %
test accuracy: 53.199999999999996 %
train accuracy: 52.28257247584139 %
test accuracy: 53.1 %
train accuracy: 52.29923358880374 %
test accuracy: 53.1 %
train accuracy: 52.28257247584139 %
test accuracy: 53.1 %
train accuracy: 52.29923358880374 %
test accuracy: 53.1 %
train accuracy: 52.34921692769077 %
test accuracy: 53.0 %
train accuracy: 52.34921692769077 %
test accuracy: 53.0 %
train accuracy: 52.415861379540154 %
test accuracy: 53.0 %
train accuracy: 52.46584471842719 %
test accuracy: 53.0 %
t

train accuracy: 53.4155281572809 %
test accuracy: 54.199999999999996 %
train accuracy: 53.448850383205595 %
test accuracy: 54.199999999999996 %
train accuracy: 53.48217260913029 %
test accuracy: 54.199999999999996 %
train accuracy: 53.46551149616794 %
test accuracy: 54.199999999999996 %
train accuracy: 53.46551149616794 %
test accuracy: 54.199999999999996 %
train accuracy: 53.43218927024325 %
test accuracy: 54.1 %
train accuracy: 53.43218927024325 %
test accuracy: 54.1 %
train accuracy: 53.43218927024325 %
test accuracy: 54.1 %
train accuracy: 53.43218927024325 %
test accuracy: 54.1 %
train accuracy: 53.43218927024325 %
test accuracy: 54.1 %
train accuracy: 53.365544818393865 %
test accuracy: 54.0 %
train accuracy: 53.365544818393865 %
test accuracy: 54.1 %
train accuracy: 53.38220593135621 %
test accuracy: 54.1 %
train accuracy: 53.38220593135621 %
test accuracy: 54.0 %
train accuracy: 53.34888370543152 %
test accuracy: 54.0 %
train accuracy: 53.33222259246917 %
test accuracy: 54.0 %


train accuracy: 54.0653115628124 %
test accuracy: 54.199999999999996 %
train accuracy: 54.04865044985005 %
test accuracy: 54.199999999999996 %
train accuracy: 54.04865044985005 %
test accuracy: 54.3 %
train accuracy: 54.08197267577474 %
test accuracy: 54.3 %
train accuracy: 54.09863378873709 %
test accuracy: 54.3 %
train accuracy: 54.09863378873709 %
test accuracy: 54.3 %
train accuracy: 54.165278240586474 %
test accuracy: 54.3 %
train accuracy: 54.19860046651117 %
test accuracy: 54.4 %
train accuracy: 54.21526157947351 %
test accuracy: 54.4 %
train accuracy: 54.23192269243585 %
test accuracy: 54.5 %
train accuracy: 54.23192269243585 %
test accuracy: 54.6 %
train accuracy: 54.21526157947351 %
test accuracy: 54.6 %
cost after  itrn 400 0.6925062680453847
train accuracy: 54.21526157947351 %
test accuracy: 54.6 %
train accuracy: 54.2485838053982 %
test accuracy: 54.5 %
train accuracy: 54.23192269243585 %
test accuracy: 54.5 %
train accuracy: 54.19860046651117 %
test accuracy: 54.5 %
train

train accuracy: 54.68177274241919 %
test accuracy: 54.699999999999996 %
train accuracy: 54.665111629456845 %
test accuracy: 54.699999999999996 %
train accuracy: 54.6484505164945 %
test accuracy: 54.699999999999996 %
train accuracy: 54.63178940353215 %
test accuracy: 54.699999999999996 %
train accuracy: 54.61512829056981 %
test accuracy: 54.699999999999996 %
train accuracy: 54.6484505164945 %
test accuracy: 54.699999999999996 %
train accuracy: 54.6484505164945 %
test accuracy: 54.8 %
train accuracy: 54.665111629456845 %
test accuracy: 54.8 %
train accuracy: 54.68177274241919 %
test accuracy: 54.8 %
train accuracy: 54.68177274241919 %
test accuracy: 54.699999999999996 %
train accuracy: 54.68177274241919 %
test accuracy: 54.699999999999996 %
train accuracy: 54.748417194268576 %
test accuracy: 54.6 %
train accuracy: 54.78173942019327 %
test accuracy: 54.6 %
train accuracy: 54.78173942019327 %
test accuracy: 54.699999999999996 %
train accuracy: 54.76507830723092 %
test accuracy: 54.69999999

...

train accuracy: 58.447184271909364 %
test accuracy: 54.5 %
train accuracy: 58.447184271909364 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.447184271909364 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.43052315894702 %
test accuracy: 54.5 %
train accuracy: 58.41386204598467 %
test accuracy: 54.5 %
train accuracy: 58.41386204598467 %
test accuracy: 54.5 %
train accuracy: 58.397200933022326 %
test accuracy: 54.5 %
train accuracy: 58.38053982005998 %
test accuracy: 54.5 %
train accuracy: 58.36387870709763 %
test accuracy: 54.5 %
train accuracy: 58.34721759413529 %
test accuracy: 54.5 %
train accuracy: 58.34721759413529 %
test accuracy: 54.5 %
train accu

train accuracy: 58.64711762745751 %
test accuracy: 55.4 %
train accuracy: 58.64711762745751 %
test accuracy: 55.4 %
train accuracy: 58.64711762745751 %
test accuracy: 55.4 %
train accuracy: 58.66377874041986 %
test accuracy: 55.4 %
train accuracy: 58.66377874041986 %
test accuracy: 55.4 %
train accuracy: 58.66377874041986 %
test accuracy: 55.4 %
train accuracy: 58.680439853382204 %
test accuracy: 55.4 %
train accuracy: 58.680439853382204 %
test accuracy: 55.4 %
train accuracy: 58.680439853382204 %
test accuracy: 55.4 %
train accuracy: 58.66377874041986 %
test accuracy: 55.3 %
train accuracy: 58.64711762745751 %
test accuracy: 55.3 %
train accuracy: 58.66377874041986 %
test accuracy: 55.3 %
train accuracy: 58.66377874041986 %
test accuracy: 55.3 %
train accuracy: 58.69710096634455 %
test accuracy: 55.3 %
train accuracy: 58.69710096634455 %
test accuracy: 55.3 %
train accuracy: 58.69710096634455 %
test accuracy: 55.3 %
train accuracy: 58.69710096634455 %
test accuracy: 55.3 %
train accur

train accuracy: 58.83038987004332 %
test accuracy: 54.699999999999996 %
train accuracy: 58.847050983005666 %
test accuracy: 54.699999999999996 %
train accuracy: 58.83038987004332 %
test accuracy: 54.6 %
train accuracy: 58.83038987004332 %
test accuracy: 54.6 %
train accuracy: 58.81372875708097 %
test accuracy: 54.6 %
train accuracy: 58.83038987004332 %
test accuracy: 54.6 %
train accuracy: 58.81372875708097 %
test accuracy: 54.6 %
train accuracy: 58.81372875708097 %
test accuracy: 54.6 %
train accuracy: 58.83038987004332 %
test accuracy: 54.699999999999996 %
train accuracy: 58.847050983005666 %
test accuracy: 54.8 %
train accuracy: 58.847050983005666 %
test accuracy: 54.8 %
train accuracy: 58.847050983005666 %
test accuracy: 54.8 %
train accuracy: 58.88037320893036 %
test accuracy: 54.8 %
train accuracy: 58.88037320893036 %
test accuracy: 54.8 %
train accuracy: 58.86371209596801 %
test accuracy: 54.8 %
train accuracy: 58.86371209596801 %
test accuracy: 54.699999999999996 %
train accura

train accuracy: 59.380206597800736 %
test accuracy: 55.1 %
train accuracy: 59.380206597800736 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.346884371876044 %
test accuracy: 55.1 %
train accuracy: 59.3302232589137 %
test accuracy: 55.1 %
train accuracy: 59.3302232589137 %
test accuracy: 55.1 %
train accuracy: 59.31356214595135 %
test accuracy: 55.1 %
train accuracy: 59.31356214595135 %
test accuracy: 55.1 %
train accuracy: 59.3302232589137 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.36354548483839 %
test accuracy: 55.1 %
train accuracy: 59.346884371876044 %
test accuracy: 55.199999999999996 %
train accuracy: 59.346884371876044 %
test accuracy: 55.199999999999996 %
train accuracy: 59.31356214595135 %
test a

train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.3302232589137 %
test accuracy: 55.4 %
train accuracy: 59.346884371876044 %
test accuracy: 55.4 %
train accuracy: 59.3302232589137 %
test accuracy: 55.4 %
train accuracy: 59.3302232589137 %
test accuracy: 55.4 %
train accuracy: 59.3302232589137 %
test accuracy: 55.4 %
train accuracy: 59.3302232589137 %
test accuracy: 55.4 %
train accuracy: 59.31356214595135 %
test accuracy: 55.4 %
train accuracy: 59.31356214595135 %
test accuracy: 55.4 %
train accuracy: 59.31356214595135 %
test accuracy: 55.4 %
train accuracy: 59.31356214595135 %
test accuracy: 55.4 %
train accuracy: 59.296901032989005 %
test accuracy: 55.4 %
train accuracy: 59.296901032989005 %
test accuracy: 55.4 %
train accur